# Step 0) Setting up the environment

In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
! pip install transformers

In [ ]:
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models, LoggingHandler, losses, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import numpy as np
import glob
import re
import os

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 1) Process Model Generation

In [ ]:
## Process model 만드는 코드 추가

In [ ]:
folder_path = "/content/drive/MyDrive/PMLAB/datasets_BPIC"
file_path_list = glob.glob(os.path.join(folder_path, "*.txt"))
groups = []

for file_path in file_path_list:
    result_list = []
    print(file_path)
    with open(file_path, 'r') as file:
        next(file)
        for line in file:
            pattern = r'"(.*?)"'
            matches = re.findall(pattern, line, flags=0)
            matches = str(matches[0]).replace("->", "\u2192")
            if matches:
                result_list.append(matches.replace('"', ""))

    groups.append(result_list)

/content/drive/MyDrive/PMLAB/datasets_BPIC/1_metadata.txt
/content/drive/MyDrive/PMLAB/datasets_BPIC/2_metadata.txt
/content/drive/MyDrive/PMLAB/datasets_BPIC/3_metadata.txt
/content/drive/MyDrive/PMLAB/datasets_BPIC/4_metadata.txt
/content/drive/MyDrive/PMLAB/datasets_BPIC/5_metadata.txt


# Step 2) Node Matching



In [ ]:
import itertools
import random

def generate_two_letter_labels():
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    labels = [''.join(pair) for pair in itertools.product(alphabet, repeat=2)]
    random.shuffle(labels)
    return labels

# 두 글자 알파벳 조합 생성
labels = generate_two_letter_labels()

In [ ]:
def find_unique_node(groups):
    unique_nodes = set()
    for group in groups:
        for item in group:
            start_index = 0
            while True:
                start_quote = item.find("'", start_index)
                if start_quote == -1:
                    break
                end_quote = item.find("'", start_quote + 1)
                if end_quote == -1:
                    break
                node = item[start_quote + 1:end_quote]
                unique_nodes.add(node)
                start_index = end_quote + 1
    return unique_nodes

unique_nodes_list = []
unique_nodes_list = find_unique_node(groups)
unique_nodes_list = list(unique_nodes_list)

for node in unique_nodes_list:
    print(node)

print(len(unique_nodes_list))

phase forwarded to competent authority
treat subcases completeness
temporary permit
enter date termination request received
create letter no permit or only notification required
phase case handled
request updated plan
phase only notification required
enter senddate letter no permit required
enter senddate decision procedure term extension
creating decision permanently suspended
suspend term
detailed procedure applies
message terminate request
calculate provisional charges
create publication document
first or second phase
publish
creating decision procedure term extension
keep permanently suspended
create message terminate request
forward transcript to stakeholders
cancel case
set decision phase no permit required decided
request complete
procedure change
retrieve missing data
enter senddate letter no permit or only notification required
read field phased application
send procedure confirmation
enter receipt of additional data
decision terminate on request
register number of days extens

In [ ]:
# 노드명과 알파벳 조합을 매칭할 딕셔너리 생성
node_to_label = {node: labels[i] for i, node in enumerate(unique_nodes_list)}

In [ ]:
# 매칭 결과 출력
for node, label in node_to_label.items():
    print(f"{node} -> {label}")

phase forwarded to competent authority -> MT
treat subcases completeness -> SL
temporary permit -> QW
enter date termination request received -> EA
create letter no permit or only notification required -> ML
phase case handled -> ZN
request updated plan -> QQ
phase only notification required -> BA
enter senddate letter no permit required -> AP
enter senddate decision procedure term extension -> XE
creating decision permanently suspended -> RX
suspend term -> AN
detailed procedure applies -> XF
message terminate request -> MY
calculate provisional charges -> VK
create publication document -> OZ
first or second phase -> WR
publish -> UX
creating decision procedure term extension -> MD
keep permanently suspended -> FW
create message terminate request -> II
forward transcript to stakeholders -> MK
cancel case -> LR
set decision phase no permit required decided -> FL
request complete -> TJ
procedure change -> AH
retrieve missing data -> HO
enter senddate letter no permit or only notificatio

In [ ]:
# 실제 데이터를 사용하여 노드명을 알파벳 조합으로 교체하는 함수
def replace_nodes_with_labels(groups, node_to_label):
    replaced_groups = []
    for group in groups:
        replaced_group = []
        for item in group:
            replaced_item = item
            for node, label in node_to_label.items():
                replaced_item = re.sub(rf"'{re.escape(node)}'", f"'{label}'", replaced_item)
            replaced_item = replaced_item.replace('tau','0')
            replaced_group.append(replaced_item)
        replaced_groups.append(replaced_group)
    return replaced_groups

In [ ]:
# 노드명을 알파벳 조합으로 교체
replaced_gropus = replace_nodes_with_labels(groups, node_to_label)

In [ ]:
all_process_trees = []

# 각 그룹에 속한 트리들을 모두 하나의 리스트로 통합
all_process_trees.extend(replaced_gropus[0])
all_process_trees.extend(replaced_gropus[1])
all_process_trees.extend(replaced_gropus[2])
all_process_trees.extend(replaced_gropus[3])
all_process_trees.extend(replaced_gropus[4])

# Step 3) GED calculation for each pair of process models

In [ ]:
pip install zss

  Preparing metadata (setup.py) ... done
  Created wheel for zss: filename=zss-1.2.0-py3-none-any.whl size=6724 sha256=82797a3b74e54b06573bf1da55bcdb61253a2ffc42183865f4e30089410a6a2e
  Stored in directory: /root/.cache/pip/wheels/f6/61/2a/cf33ab7301cc318a13418d9a805c1832be561b46e7d9337625
Successfully built zss


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from zss import simple_distance, distance
from zss import Node
from tqdm import tqdm

class TreeNode:
    def __init__(self, value):
        self.value = value
        self.children = []

    def add_child(self, child):
        self.children.append(child)

    def print_tree(self, depth=0):
        print("  " * depth + self.value)
        for child in self.children:
            child.print_tree(depth + 1)

    def to_zss_node(self):
        children_nodes = [child.to_zss_node() for child in self.children]
        return Node(self.value, children_nodes)

    def count_nodes_and_edges(self):
        # 노드와 엣지 수를 동시에 계산
        count_nodes = 1
        count_edges = len(self.children)
        for child in self.children:
            nodes, edges = child.count_nodes_and_edges()
            count_nodes += nodes
            count_edges += edges
        return count_nodes, count_edges

def text_to_tree(sentence):
    sentence = sentence.replace(" ", "").replace(",", "")
    stack = []
    root = TreeNode(None)
    i = 0
    while i < len(sentence):
        if sentence[i] != '(' and sentence[i] != ')':
            if sentence[i] == "'":
                j = i + 1
                while sentence[j] != "'":
                    j += 1
                node = sentence[i + 1:j]
                child = TreeNode(node)
                root.add_child(child)
                i = j + 1
            else:
                child = TreeNode(sentence[i])
                root.add_child(child)
                i += 1
        elif sentence[i] == '(':
            stack.append(child)
            root = child
            i += 1
        else:
            stack.pop()
            if stack:
                root = stack[-1]
            i += 1
    return root

def tree_edit_distance(graph1, graph2):
    tree1 = graph1.to_zss_node()
    tree2 = graph2.to_zss_node()
    return simple_distance(tree1, tree2)

def max_edit_distance(graph1, graph2):
    nodes1, edges1 = graph1.count_nodes_and_edges()
    nodes2, edges2 = graph2.count_nodes_and_edges()
    return nodes1 + edges1 + nodes2 + edges2

def graph_edit_similarity(sentence1, sentence2):
    graph1 = text_to_tree(sentence1)
    graph2 = text_to_tree(sentence2)
    ged = tree_edit_distance(graph1, graph2)
    maxged = max_edit_distance(graph1, graph2)

    return 1 - (ged / maxged)

def ged_average_similarity(tree_group1, tree_group2):
    total_similarity = 0.0
    total_pairs = len(tree_group1) * len(tree_group2)

    for tree1 in tree_group1:
        for tree2 in tree_group2:
            total_similarity += graph_edit_similarity(tree1, tree2)

    return total_similarity / total_pairs

def make_dataset(pts):
    data = []
    total_iterations = len(pts) * (len(pts) + 1) // 2
    progress_bar = tqdm(total=total_iterations, desc='Progress', unit=' pairs')

    for i in range(len(pts)):
        for j in range(i, len(pts)):
            seq1 = pts[i]
            seq2 = pts[j]
            similarity = graph_edit_similarity(seq1, seq2)
            new_data = InputExample(texts=[seq1, seq2], label=similarity)
            data.append(new_data)
            progress_bar.update(1)

    progress_bar.close()
    return data

In [ ]:
import random

# 데이터셋의 크기와 1퍼센트에 해당하는 샘플 개수 계산
total_samples = len(all_process_trees)
one_percent_samples = int(total_samples * 0.1)

# 무작위로 1퍼센트의 샘플 선택
random_samples = random.sample(all_process_trees, one_percent_samples)

# 선택된 샘플 확인
print("선택된 샘플 수:", len(random_samples))

선택된 샘플 수: 65


In [ ]:
datasets = make_dataset(random_samples)

Progress:   3%|▎         | 72/2145 [02:58<8:52:39, 15.42s/ pairs] Exception ignored in: <function _xla_gc_callback at 0x79f5021af250>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
Progress:   3%|▎         | 73/2145 [03:48<14:49:28, 25.76s/ pairs]Exception ignored in: <function _xla_gc_callback at 0x79f5021af250>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x79f5021af250>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
import pickle

# pkl 파일로 저장
with open('datasets2.pkl', 'wb') as f:
    pickle.dump(datasets, f)

# Step 4) Fine-tuning the SBERT model based on GED similarity

In [ ]:
import pickle

# pkl 파일에서 데이터셋 로드
with open('/content/drive/MyDrive/PMLAB/datasets2.pkl', 'rb') as f:
    datasets = pickle.load(f)

In [ ]:
for i in range(len(datasets)):
    datasets[i].label = np.float32(datasets[i].label)

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋을 train과 나머지로 나눔
train_data, temp_data = train_test_split(datasets, test_size=0.2, random_state=42)

# 나머지를 test와 eval로 나눔 (전체 데이터셋의 20%를 각각의 test와 eval로 사용)
test_data, eval_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# Train Dataloader
train_dataloader = DataLoader(
    train_data,
    shuffle=True,
    batch_size=16,
)

# Evaluator by sts-validation
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    eval_data,
    name="sts-dev",
)

# Evaluator by sts-test
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    test_data,
    name="sts-test",
)


In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2').to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Use CosineSimilarityLoss
train_loss = losses.CosineSimilarityLoss(model=sbert_model)
sbert_model.to(device)

# warmup steps
warmup_steps = math.ceil(len(datasets) * 4 / 16 * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Training
sbert_model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=dev_evaluator,
    epochs=4,
    evaluation_steps=int(len(train_dataloader)*0.1),
    warmup_steps=warmup_steps
)

Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine,Sts-dev Pearson Manhattan,Sts-dev Spearman Manhattan,Sts-dev Pearson Euclidean,Sts-dev Spearman Euclidean,Sts-dev Pearson Dot,Sts-dev Spearman Dot,Sts-dev Pearson Max,Sts-dev Spearman Max
10,No log,No log,0.540968,0.427256,0.646829,0.427600,0.649042,0.427256,0.540968,0.427256,0.649042,0.427600
20,No log,No log,0.467404,0.269704,0.592956,0.268010,0.598549,0.269704,0.467404,0.269707,0.598549,0.269707
30,No log,No log,0.646308,0.484916,0.729041,0.484429,0.733371,0.484916,0.646308,0.484916,0.733371,0.484916
40,No log,No log,0.618048,0.450131,0.700377,0.444660,0.707245,0.450131,0.618048,0.450130,0.707245,0.450131
50,No log,No log,0.637267,0.465517,0.710015,0.458041,0.718211,0.465517,0.637267,0.465514,0.718211,0.465517
60,No log,No log,0.751341,0.603986,0.801285,0.600273,0.808205,0.603986,0.751341,0.603981,0.808205,0.603986
70,No log,No log,0.755199,0.601901,0.808704,0.597617,0.814292,0.601901,0.755199,0.601896,0.814292,0.601901
80,No log,No log,0.771052,0.626194,0.819676,0.625498,0.824774,0.626194,0.771052,0.626195,0.824774,0.626195
90,No log,No log,0.698863,0.520933,0.756738,0.522353,0.760106,0.520933,0.698863,0.520932,0.760106,0.522353
100,No log,No log,0.786335,0.666675,0.828568,0.662362,0.832104,0.666675,0.786335,0.666674,0.832104,0.666675


In [ ]:
# 모델 저장
output_path = "/content/drive/MyDrive/PMLAB/sbert_fine_tuned_BPIC"
sbert_model.save(output_path)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

# + Levenshtine Fine-tuning

In [ ]:
pip install levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 63.2 MB/s eta 0:00:00


In [ ]:
# 각 문자열 요소에 대해 replace를 적용
cleaned_all = [item.replace("'", "").replace(",", "") for item in all_process_trees]

In [ ]:
## (tree1, tree2, levenshtein sim)
import Levenshtein
from sentence_transformers.readers import InputExample

def make_dataset(pts):

    data = []
    total_iterations = len(pts) * (len(pts) + 1) // 2  # 전체 반복 횟수 계산

    for i in range(len(pts)):
        for j in range(i, len(pts)):
            seq1 = pts[i]
            seq2 = pts[j]
            distance = Levenshtein.distance(seq1, seq2)
            similarity = 1 - distance / max(len(seq1), len(seq2))
            new_data = InputExample(texts=[seq1, seq2], label=similarity)
            data.append(new_data)

    return data

datasets = make_dataset(cleaned_all)

In [ ]:
import random

# 데이터셋의 크기와 1퍼센트에 해당하는 샘플 개수 계산
total_samples = len(datasets)
one_percent_samples = int(total_samples * 0.01)

# 무작위로 1퍼센트의 샘플 선택
random_samples = random.sample(datasets, one_percent_samples)

# 선택된 샘플 확인
print("선택된 샘플 수:", len(random_samples))

선택된 샘플 수: 2135


In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋을 train과 나머지로 나눔
train_data, temp_data = train_test_split(random_samples, test_size=0.2, random_state=42)

# 나머지를 test와 eval로 나눔 (전체 데이터셋의 20%를 각각의 test와 eval로 사용)
test_data, eval_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

# Train Dataloader
train_dataloader = DataLoader(
    train_data,
    shuffle=True,
    batch_size=32,
)

# Evaluator by sts-validation
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    eval_data,
    name="sts-dev",
)

# Evaluator by sts-test
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    test_data,
    name="sts-test",
)

In [ ]:
from sentence_transformers import SentenceTransformer, models

# Load Embedding Model
embedding_model = models.Transformer(
    model_name_or_path="deepset/roberta-base-squad2",
    max_seq_length=256,
    do_lower_case=True
)

# Only use Mean Pooling -> Pooling all token embedding vectors of sentence.
pooling_model = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooling_model])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Use CosineSimilarityLoss
train_loss = losses.CosineSimilarityLoss(model=model)
model.to(device)

# warmup steps
warmup_steps = math.ceil(len(datasets) * 4 / 32 * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Training
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=dev_evaluator,
    epochs=4,
    evaluation_steps=int(len(train_dataloader)*0.1),
    warmup_steps=warmup_steps
)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/54 [00:00<?, ?it/s]

Iteration:   0%|          | 0/54 [00:00<?, ?it/s]

Iteration:   0%|          | 0/54 [00:00<?, ?it/s]

Iteration:   0%|          | 0/54 [00:00<?, ?it/s]

# Embedding

In [ ]:
def tokenize_tree(tree):
    # 특수 문자를 이스케이프하여 정규 표현식 적용 + 괄호는 넣기
    special_characters = r"[\*\+\./:;<=\?\[\]\^_`{|}~\(\)]"
    tokenized_tree = re.findall(r"\b\w+\b|" + special_characters + r"|\u2192", str(tree))
    return tokenized_tree

In [ ]:
def embed_processes_with_sbert(tokenized_processes):

    # 임베딩을 저장할 리스트 초기화
    sbert_embeddings = []

    # tokenized_processes에 대한 반복문
    for tree_tokens in tokenized_processes:
        # 프로세스 트리를 텍스트로 펼치고 토큰화
        process_tree_text = " ".join(tree_tokens)

        # SBERT 모델을 사용하여 문장 임베딩 얻기
        sbert_embedding = sbert_model.encode([process_tree_text])

        # 평균 풀링
        average_pooled_embedding = np.mean(sbert_embedding, axis=0)
        sbert_embeddings.append(average_pooled_embedding)

    # 모든 프로세스 트리에 대한 임베딩이 sbert_embeddings에 저장됨
    return sbert_embeddings

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def cosine_avg(embeddings_groups):
    num_groups = len(embeddings_groups)
    similarity_matrix = np.zeros((num_groups, num_groups))

    for i in range(num_groups):
        for j in range(i, num_groups):
            embeddings1 = embeddings_groups[i]
            embeddings2 = embeddings_groups[j]

            similarity_matrix[i, j] = np.mean(cosine_similarity(embeddings1, embeddings2))

    # Print the similarity matrix
    print("Similarity Matrix:")
    print(similarity_matrix)

## Embedding (rev_17~)

In [ ]:
folder_path = "/content/drive/MyDrive/PMLAB/datasets_BPIC_2"
file_path_list = glob.glob(os.path.join(folder_path, "*.txt"))
result_lists = []

for file_path in file_path_list:
    result_list = []
    print(file_path)
    with open(file_path, 'r') as file:
        next(file)
        for line in file:
            pattern = r'"(.*?)"'
            matches = re.findall(pattern, line, flags=0)
            matches = str(matches[0]).replace("->", "\u2192")
            if matches:
                result_list.append(matches.replace('"', ""))

    result_lists.append(result_list)

BPIC15_1_rev = result_lists[2]
BPIC15_2_rev = result_lists[0]
BPIC15_3_rev = result_lists[3]
BPIC15_4_rev = result_lists[4]
BPIC15_5_rev = result_lists[1]

/content/drive/MyDrive/PMLAB/datasets_BPIC_2/2_metadata.txt
/content/drive/MyDrive/PMLAB/datasets_BPIC_2/5_metadata.txt
/content/drive/MyDrive/PMLAB/datasets_BPIC_2/1_metadata.txt
/content/drive/MyDrive/PMLAB/datasets_BPIC_2/3_metadata.txt
/content/drive/MyDrive/PMLAB/datasets_BPIC_2/4_metadata.txt


In [ ]:
groups_rev = [BPIC15_1_rev,BPIC15_2_rev,BPIC15_3_rev,BPIC15_4_rev,BPIC15_5_rev]

In [ ]:
# Tokenize each group
tokenized_1_rev = [tokenize_tree(tree) for tree in groups_rev[0]]
tokenized_2_rev = [tokenize_tree(tree) for tree in groups_rev[1]]
tokenized_3_rev = [tokenize_tree(tree) for tree in groups_rev[2]]
tokenized_4_rev = [tokenize_tree(tree) for tree in groups_rev[3]]
tokenized_5_rev = [tokenize_tree(tree) for tree in groups_rev[4]]

In [ ]:
tokenized_all = []

# 토큰화된 트리들을 각 그룹에서 가져와서 통합
tokenized_all.extend(tokenized_1_rev)
tokenized_all.extend(tokenized_2_rev)
tokenized_all.extend(tokenized_3_rev)
tokenized_all.extend(tokenized_4_rev)
tokenized_all.extend(tokenized_5_rev)
tokenized_group = [tokenized_1_rev, tokenized_2_rev, tokenized_3_rev, tokenized_4_rev, tokenized_5_rev]

In [ ]:
SSL_sbert_embeddings_1 = []
SSL_sbert_embeddings_2 = []
SSL_sbert_embeddings_3 = []
SSL_sbert_embeddings_4 = []
SSL_sbert_embeddings_5 = []

for idx, group in enumerate(tokenized_group):
    sbert_embeddings = embed_processes_with_sbert(group)
    exec(f"SSL_sbert_embeddings_{idx+1} = sbert_embeddings")

In [ ]:
embeddings_groups_sbert = [SSL_sbert_embeddings_1, SSL_sbert_embeddings_2, SSL_sbert_embeddings_3, SSL_sbert_embeddings_4, SSL_sbert_embeddings_5]
cosine_avg(embeddings_groups_sbert)
# Levenshtein 2400

Similarity Matrix:
[[0.86841255 0.77147526 0.79351217 0.85059887 0.81059641]
 [0.         0.80953777 0.79787397 0.80141276 0.76866233]
 [0.         0.         0.79905343 0.81235725 0.77934176]
 [0.         0.         0.         0.85877079 0.81387043]
 [0.         0.         0.         0.         0.78294343]]


In [ ]:
embeddings_groups_sbert = [SSL_sbert_embeddings_1, SSL_sbert_embeddings_2, SSL_sbert_embeddings_3, SSL_sbert_embeddings_4, SSL_sbert_embeddings_5]
cosine_avg(embeddings_groups_sbert)
# Levenshtein 2400 rev

Similarity Matrix:
[[0.9183225  0.8817445  0.90072441 0.91033047 0.88654429]
 [0.         0.90099102 0.89342874 0.87888408 0.8663978 ]
 [0.         0.         0.90475243 0.89721817 0.88070369]
 [0.         0.         0.         0.91224414 0.88424754]
 [0.         0.         0.         0.         0.8678332 ]]


In [ ]:
embeddings_groups_sbert = [SSL_sbert_embeddings_1, SSL_sbert_embeddings_2, SSL_sbert_embeddings_3, SSL_sbert_embeddings_4, SSL_sbert_embeddings_5]
cosine_avg(embeddings_groups_sbert)
# Levenshtein 2100

Similarity Matrix:
[[0.6770606  0.56850946 0.61906612 0.58296448 0.49907029]
 [0.         0.62727594 0.61145461 0.5916571  0.56734091]
 [0.         0.         0.62478495 0.59466916 0.54165632]
 [0.         0.         0.         0.57469547 0.53484815]
 [0.         0.         0.         0.         0.53568292]]


In [ ]:
embeddings_groups_sbert = [SSL_sbert_embeddings_1, SSL_sbert_embeddings_2, SSL_sbert_embeddings_3, SSL_sbert_embeddings_4, SSL_sbert_embeddings_5]
cosine_avg(embeddings_groups_sbert)
# BPIC_rev

Similarity Matrix:
[[0.90015835 0.88644475 0.8903569  0.87843186 0.84922892]
 [0.         0.90839881 0.89664835 0.88340008 0.87319165]
 [0.         0.         0.89360231 0.8814404  0.86239433]
 [0.         0.         0.         0.8740527  0.85546827]
 [0.         0.         0.         0.         0.8578068 ]]


In [ ]:
embeddings_groups_sbert = [SSL_sbert_embeddings_1, SSL_sbert_embeddings_2, SSL_sbert_embeddings_3, SSL_sbert_embeddings_4, SSL_sbert_embeddings_5]
cosine_avg(embeddings_groups_sbert)
# BPIC 기본

Similarity Matrix:
[[0.87214839 0.84303463 0.85530585 0.84184247 0.81481129]
 [0.         0.87786186 0.87119275 0.85163647 0.84920847]
 [0.         0.         0.87281114 0.85169339 0.83961129]
 [0.         0.         0.         0.83750087 0.82620174]
 [0.         0.         0.         0.         0.83132523]]


In [ ]:
embeddings_groups_sbert = [SSL_sbert_embeddings_1, SSL_sbert_embeddings_2, SSL_sbert_embeddings_3, SSL_sbert_embeddings_4, SSL_sbert_embeddings_5]
cosine_avg(embeddings_groups_sbert)
# Levenshtein 2400 rev

Similarity Matrix:
[[0.92921722 0.91471332 0.90271759 0.90512341 0.86346066]
 [0.         0.91617125 0.8946054  0.90298653 0.87554955]
 [0.         0.         0.88164634 0.88554627 0.85065979]
 [0.         0.         0.         0.89479953 0.86528635]
 [0.         0.         0.         0.         0.8548004 ]]


## Embedding (noise 20-80)

In [ ]:
folder_path = "/content/drive/MyDrive/PMLAB/noise_20_80/20"
file_path_list = glob.glob(os.path.join(folder_path, "*.txt"))
result_lists = []

for file_path in file_path_list:
    result_list = []
    print(file_path)
    with open(file_path, 'r') as file:
        next(file)
        for line in file:
            pattern = r'"(.*?)"'
            matches = re.findall(pattern, line, flags=0)
            matches = str(matches[0]).replace("->", "\u2192")
            if matches:
                result_list.append(matches.replace('"', ""))

    result_lists.append(result_list)

BPIC15_1_rev = result_lists[0]
BPIC15_2_rev = result_lists[1]
BPIC15_3_rev = result_lists[2]
BPIC15_4_rev = result_lists[3]
BPIC15_5_rev = result_lists[4]

/content/drive/MyDrive/PMLAB/noise_20_80/20/1_metadata.txt
/content/drive/MyDrive/PMLAB/noise_20_80/20/2_metadata.txt
/content/drive/MyDrive/PMLAB/noise_20_80/20/3_metadata.txt
/content/drive/MyDrive/PMLAB/noise_20_80/20/4_metadata.txt
/content/drive/MyDrive/PMLAB/noise_20_80/20/5_metadata.txt


In [ ]:
groups_rev = [BPIC15_1_rev,BPIC15_2_rev,BPIC15_3_rev,BPIC15_4_rev,BPIC15_5_rev]

In [ ]:
def tokenize_tree(tree):
    # 특수 문자를 이스케이프하여 정규 표현식 적용 + 괄호는 넣기
    special_characters = r"[\*\+\./:;<=\?\[\]\^_`{|}~\(\)]"
    tokenized_tree = re.findall(r"\b\w+\b|" + special_characters + r"|\u2192", str(tree))
    return tokenized_tree

# Tokenize each group
tokenized_1_rev = [tokenize_tree(tree) for tree in groups_rev[0]]
tokenized_2_rev = [tokenize_tree(tree) for tree in groups_rev[1]]
tokenized_3_rev = [tokenize_tree(tree) for tree in groups_rev[2]]
tokenized_4_rev = [tokenize_tree(tree) for tree in groups_rev[3]]
tokenized_5_rev = [tokenize_tree(tree) for tree in groups_rev[4]]

In [ ]:
tokenized_all = []

# 토큰화된 트리들을 각 그룹에서 가져와서 통합
tokenized_all.extend(tokenized_1_rev)
tokenized_all.extend(tokenized_2_rev)
tokenized_all.extend(tokenized_3_rev)
tokenized_all.extend(tokenized_4_rev)
tokenized_all.extend(tokenized_5_rev)
tokenized_group = [tokenized_1_rev, tokenized_2_rev, tokenized_3_rev, tokenized_4_rev, tokenized_5_rev]

In [ ]:
def embed_processes_with_sbert(tokenized_processes):

    # 임베딩을 저장할 리스트 초기화
    sbert_embeddings = []

    # tokenized_processes에 대한 반복문
    for tree_tokens in tokenized_processes:
        # 프로세스 트리를 텍스트로 펼치고 토큰화
        process_tree_text = " ".join(tree_tokens)

        # SBERT 모델을 사용하여 문장 임베딩 얻기
        sbert_embedding = sbert_model.encode([process_tree_text])

        # 평균 풀링
        average_pooled_embedding = np.mean(sbert_embedding, axis=0)
        sbert_embeddings.append(average_pooled_embedding)

    # 모든 프로세스 트리에 대한 임베딩이 sbert_embeddings에 저장됨
    return sbert_embeddings

In [ ]:
SSL_sbert_embeddings_1 = []
SSL_sbert_embeddings_2 = []
SSL_sbert_embeddings_3 = []
SSL_sbert_embeddings_4 = []
SSL_sbert_embeddings_5 = []

for idx, group in enumerate(tokenized_group):
    sbert_embeddings = embed_processes_with_sbert(group)
    exec(f"SSL_sbert_embeddings_{idx+1} = sbert_embeddings")

In [ ]:
embeddings_groups_sbert = [SSL_sbert_embeddings_1, SSL_sbert_embeddings_2, SSL_sbert_embeddings_3, SSL_sbert_embeddings_4, SSL_sbert_embeddings_5]
cosine_avg(embeddings_groups_sbert)

Similarity Matrix:
[[0.84821451 0.75165647 0.7784009  0.79631227 0.79072511]
 [0.         0.82587659 0.77918261 0.78016067 0.78734159]
 [0.         0.         0.83678091 0.79080117 0.78255403]
 [0.         0.         0.         0.84104347 0.81185061]
 [0.         0.         0.         0.         0.84802794]]


## Embedding (length 17~22, noise 5filter)

In [ ]:
# 폴더 경로 설정
folder_path = "/content/drive/MyDrive/PMLAB/length_17_22/noise_filter"
file_path_list = glob.glob(os.path.join(folder_path, "*.txt"))
result_lists = []

# 파일 경로 출력 및 존재 여부 확인
print("Processing the following files:")
for file_path in file_path_list:
    print(file_path)
    if not os.path.exists(file_path):
        print(f"File does not exist: {file_path}")
        continue  # 파일이 존재하지 않으면 다음 파일로 넘어감

    result_list = []
    with open(file_path, 'r') as file:
        for line in file:
            pattern = r'"(.*?)"'
            matches = re.findall(pattern, line)
            if matches:  # matches 리스트가 비어 있지 않은 경우에만 처리
                match = matches[0].replace("->", "\u2192")
                result_list.append(match.replace('"', ""))

    result_lists.append(result_list)

BPIC_1_5 = result_lists[0]
BPIC_6_10 = result_lists[1]
BPIC_11_15 = result_lists[2]
BPIC_16_20 = result_lists[3]
BPIC_21_25 = result_lists[4]
BPIC_26_30 = result_lists[5]
BPIC_31_35 = result_lists[6]
BPIC_36_40 = result_lists[7]
BPIC_41_45 = result_lists[8]
BPIC_46_50 = result_lists[9]
BPIC_51_55 = result_lists[10]
BPIC_56_60 = result_lists[11]
BPIC_61_65 = result_lists[12]
BPIC_66_70 = result_lists[13]
BPIC_71_75 = result_lists[19]
BPIC_76_80 = result_lists[14]
BPIC_81_85 = result_lists[15]
BPIC_86_90 = result_lists[16]
BPIC_91_95 = result_lists[18]
BPIC_96_100 = result_lists[17]

Processing the following files:
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/1_5.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/6_10.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/11_15.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/16_20.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/21_25.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/26_30.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/31_35.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/36_40.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/41_45.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/46_50.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/51_55.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/56_60.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/61_65.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/66_70.txt
/content/drive/MyDrive/PMLAB/length_17_22/noise_filter/76_80.

In [ ]:
groups_17_20 = [
    BPIC_1_5, BPIC_6_10, BPIC_11_15, BPIC_16_20, BPIC_21_25,
    BPIC_26_30, BPIC_31_35, BPIC_36_40, BPIC_41_45, BPIC_46_50,
    BPIC_51_55, BPIC_56_60, BPIC_61_65, BPIC_66_70, BPIC_71_75,
    BPIC_76_80, BPIC_81_85, BPIC_86_90, BPIC_91_95, BPIC_96_100
]

In [ ]:
# Tokenize each group'
tokenized_group = []
for i in range(len(groups_17_20)):
    tokenized_group.append([tokenize_tree(tree) for tree in groups_17_20[i]])

In [ ]:
embeddings_groups_sbert = []
for i in range(len(tokenized_group)):
    f'SSL_sbert_embeddings_{i} = []'
    f'embeddings_groups_sbert.append(SSL_sbert_embeddings_{i})'

for idx, group in enumerate(tokenized_group):
    sbert_embeddings = embed_processes_with_sbert(group)
    exec(f"embeddings_groups_sbert.append(sbert_embeddings)")

In [ ]:
cosine_avg(embeddings_groups_sbert)

Similarity Matrix:
[[0.85761756 0.85110533 0.82984179 0.83161658 0.83171958 0.84234899
  0.80132216 0.79870909 0.81376779 0.82390565 0.82323343 0.81387168
  0.83158022 0.81929749 0.7930221  0.83204818 0.80002689 0.82207644
  0.79690325 0.7824561 ]
 [0.         0.8524704  0.83062625 0.83228368 0.83138347 0.84138018
  0.799505   0.79711121 0.81303877 0.82238787 0.82118672 0.81028646
  0.82587916 0.81716388 0.7951318  0.83224708 0.80139804 0.81772107
  0.79499519 0.77871239]
 [0.         0.         0.82638949 0.8269496  0.82555747 0.82938468
  0.7969259  0.79138798 0.80294925 0.81175798 0.80575681 0.79609644
  0.80966878 0.80501199 0.78043634 0.81996393 0.78636241 0.80704278
  0.78417879 0.76776367]
 [0.         0.         0.         0.83862728 0.83221442 0.83632606
  0.80384719 0.798343   0.80337578 0.81435394 0.80364174 0.79896933
  0.81056184 0.8091318  0.7822296  0.82260811 0.7874977  0.81140363
  0.7905249  0.77354103]
 [0.         0.         0.         0.         0.83759087 0.843468

## Embedding 17_22

In [ ]:
folder_path = "/content/drive/MyDrive/PMLAB/length_17_22"
file_path_list = glob.glob(os.path.join(folder_path, "*.txt"))
result_lists = []

for file_path in file_path_list:
    result_list = []
    print(file_path)
    with open(file_path, 'r') as file:
        next(file)
        for line in file:
            pattern = r'"(.*?)"'
            matches = re.findall(pattern, line, flags=0)
            matches = str(matches[0]).replace("->", "\u2192")
            if matches:
                result_list.append(matches.replace('"', ""))

    result_lists.append(result_list)

BPIC15_1_rev = result_lists[0]
BPIC15_2_rev = result_lists[1]
BPIC15_3_rev = result_lists[2]
BPIC15_4_rev = result_lists[3]
BPIC15_5_rev = result_lists[4]

In [ ]:
groups_rev = [BPIC15_1_rev,BPIC15_2_rev,BPIC15_3_rev,BPIC15_4_rev,BPIC15_5_rev]

In [ ]:
# Tokenize each group
tokenized_1_rev = [tokenize_tree(tree) for tree in groups_rev[0]]
tokenized_2_rev = [tokenize_tree(tree) for tree in groups_rev[1]]
tokenized_3_rev = [tokenize_tree(tree) for tree in groups_rev[2]]
tokenized_4_rev = [tokenize_tree(tree) for tree in groups_rev[3]]
tokenized_5_rev = [tokenize_tree(tree) for tree in groups_rev[4]]

In [ ]:
tokenized_all = []

# 토큰화된 트리들을 각 그룹에서 가져와서 통합
tokenized_all.extend(tokenized_1_rev)
tokenized_all.extend(tokenized_2_rev)
tokenized_all.extend(tokenized_3_rev)
tokenized_all.extend(tokenized_4_rev)
tokenized_all.extend(tokenized_5_rev)
tokenized_group = [tokenized_1_rev, tokenized_2_rev, tokenized_3_rev, tokenized_4_rev, tokenized_5_rev]

In [ ]:
SSL_sbert_embeddings_1 = []
SSL_sbert_embeddings_2 = []
SSL_sbert_embeddings_3 = []
SSL_sbert_embeddings_4 = []
SSL_sbert_embeddings_5 = []

for idx, group in enumerate(tokenized_group):
    sbert_embeddings = embed_processes_with_sbert(group)
    exec(f"SSL_sbert_embeddings_{idx+1} = sbert_embeddings")

In [ ]:
embeddings_groups_sbert = [SSL_sbert_embeddings_1, SSL_sbert_embeddings_2, SSL_sbert_embeddings_3, SSL_sbert_embeddings_4, SSL_sbert_embeddings_5]
cosine_avg(embeddings_groups_sbert)
# Levenshtein 2400

Similarity Matrix:
[[0.9303844  0.73868316 0.8038556  0.85501891 0.79312074]
 [0.         0.91070682 0.75812483 0.75322622 0.74592394]
 [0.         0.         0.92482728 0.80968368 0.7675162 ]
 [0.         0.         0.         0.92602223 0.81886256]
 [0.         0.         0.         0.         0.8801676 ]]
